<a href="https://colab.research.google.com/github/barbarajunq131/Programas/blob/main/MODELAGEM_ESTAT%C3%8DSTICA__Exercicio__Lista_1_B%C3%A1rbaraJunqueira.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Disciplina : Modelagem Estatística - T01 (2025.2) /n

Aluna: Bárbara Junqueira dos Santos /n

Matricula: 20252001826 /n

Lista de exercício: 1/n

link do colab: https://colab.research.google.com/drive/1O5hS-31GQPMuhSdmfuYdh5AhF-ACARCJ#scrollTo=PQXmsPBUL6z9

In [19]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from statsmodels.stats.diagnostic import het_breuschpagan
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
import matplotlib as mpl
import textwrap

In [20]:
PATH_BANCO = '/content/banco.csv'
OUTPUT_PDF = '/content/lista1_report_detalhado.pdf'
FONT_SIZE = 10
MAX_PAGES = 10

In [21]:
mpl.rcParams['font.family'] = 'sans-serif'
mpl.rcParams['font.sans-serif'] = ['DejaVu Sans', 'Arial', 'Helvetica', 'Liberation Sans']
mpl.rcParams['font.size'] = FONT_SIZE

def wrap_lines(text, width=95):
    return textwrap.wrap(text, width)

def page_text(pdf, title, lines):
    fig, ax = plt.subplots(figsize=(8.27, 11.69))
    ax.axis('off')
    y = 0.98
    ax.text(0.02, y, title, fontsize=12, weight='bold', transform=fig.transFigure)
    y -= 0.04
    for line in lines:
        wrapped = wrap_lines(line, width=95)
        for sub in wrapped:
            ax.text(0.02, y, sub, fontsize=FONT_SIZE, transform=fig.transFigure)
            y -= 0.025
            if y < 0.02:
                break
        if y < 0.02:
            break
    pdf.savefig(fig)
    plt.close(fig)


In [22]:
mpl.rcParams['font.family'] = 'sans-serif'
mpl.rcParams['font.sans-serif'] = ['DejaVu Sans', 'Arial', 'Helvetica', 'Liberation Sans']
mpl.rcParams['font.size'] = FONT_SIZE

def wrap_lines(text, width=95):
    return textwrap.wrap(text, width)

def page_text(pdf, title, lines):
    fig, ax = plt.subplots(figsize=(8.27, 11.69))
    ax.axis('off')
    y = 0.98
    ax.text(0.02, y, title, fontsize=12, weight='bold', transform=fig.transFigure)
    y -= 0.04
    for line in lines:
        wrapped = wrap_lines(line, width=95)
        for sub in wrapped:
            ax.text(0.02, y, sub, fontsize=FONT_SIZE, transform=fig.transFigure)
            y -= 0.025
            if y < 0.02:
                break
        if y < 0.02:
            break
    pdf.savefig(fig)
    plt.close(fig)

In [23]:
banco = pd.read_csv(PATH_BANCO)

In [24]:
if 'instrucao' in banco.columns:
    banco['instrucao'] = banco['instrucao'].astype('category')
if 'ramo' in banco.columns:
    banco['ramo'] = banco['ramo'].astype('category')

# ==========================
# Fórmula base
# ==========================
formula_prof = '''
    logsalh ~ idade + I(idade**2) + mulher
    + C(instrucao, Treatment(reference=1))
    + C(ramo, Treatment(reference=1))
    + rm + nao_branco + urbano + form
'''

QUESTÃO 1 - Regressão Simples: Educação e Salário

In [25]:
q1_code = [
    "CÓDIGO:",
    "modelo_q1 = smf.ols('logsalh ~ C(instrucao)', data=banco).fit()",
    "print(modelo_q1.summary())"
]

modelo_q1 = smf.ols('logsalh ~ C(instrucao)', data=banco).fit()

q1_lines = q1_code + [""] + [
    "RESULTADOS E INTERPRETAÇÃO:",
    "",
    "1.a) Os coeficientes associados à instrução são significativos?",
    f"- Referência: instrucao=1 (Fundamental incompleto/sem instrução)",
    ""
]

# Coletar coeficientes significativos
significativos = []
nao_significativos = []
for name, coef in modelo_q1.params.items():
    p = modelo_q1.pvalues[name]
    if p < 0.05:
        significativos.append(f"{name}: coef={coef:.4f} (p={p:.4f})")
    else:
        nao_significativos.append(f"{name}: coef={coef:.4f} (p={p:.4f})")

q1_lines.append("Coeficientes SIGNIFICATIVOS (p < 0.05):")
for sig in significativos:
    q1_lines.append(f"  • {sig}")

if nao_significativos:
    q1_lines.append("")
    q1_lines.append("Coeficientes NÃO SIGNIFICATIVOS:")
    for ns in nao_significativos:
        q1_lines.append(f"  • {ns}")

q1_lines.extend([
    "",
    "1.b) O que significa, em termos salariais, a diferença entre níveis de educação?",
    "- Coeficientes representam diferenças em LOG salário-hora em relação à categoria base",
    "- Para converter em %: (exp(coef) - 1) * 100",
    ""
])

# Calcular diferenças percentuais
for name, coef in modelo_q1.params.items():
    if 'instrucao' in name:
        perc = (np.exp(coef) - 1) * 100
        nivel = name.split('[')[-1].replace(']', '') if '[' in name else name
        q1_lines.append(f"  {nivel}: {perc:+.1f}% em relação ao fundamental incompleto")

q1_lines.extend([
    "",
    "CONCLUSÃO: Educação tem forte relação com salários, com gradiente positivo claro.",
    "Quanto maior a escolaridade, maior o salário-hora, em média."
])


QUESTÃO 2 – Regressão Múltipla

In [26]:
q2_code = [
    "CÓDIGO:",
    "modelo_q2 = smf.ols(formula=formula_prof, data=banco).fit()",
    "print(modelo_q2.summary())"
]

modelo_q2 = smf.ols(formula=formula_prof, data=banco).fit()

q2_lines = q2_code + [""] + [
    "RESULTADOS E INTERPRETAÇÃO:",
    "",
    "2.a) Compare o coeficiente de instrucao aqui com o da regressão da Questão 1:",
    ""
]

# Comparar coeficientes de instrução
for i in range(2, 8):  # instrucao2 a instrucao7
    nome_coef_q1 = f'C(instrucao)[T.{i}]'
    nome_coef_q2 = f'C(instrucao, Treatment(reference=1))[T.{i}]'

    coef_q1 = modelo_q1.params.get(nome_coef_q1, np.nan)
    coef_q2 = modelo_q2.params.get(nome_coef_q2, np.nan)

    if not np.isnan(coef_q1) and not np.isnan(coef_q2):
        diff = coef_q2 - coef_q1
        diff_pct = (diff / coef_q1) * 100 if coef_q1 != 0 else 0
        q2_lines.append(f"  Instrução {i}:")
        q2_lines.append(f"    Q1 (simples) = {coef_q1:.4f}")
        q2_lines.append(f"    Q2 (múltipla) = {coef_q2:.4f}")
        q2_lines.append(f"    Diferença = {diff:+.4f} ({diff_pct:+.1f}%)")

q2_lines.extend([
    "",
    "2.b) Por que houve diferença?",
    "- Na regressão simples, o efeito da educação captura também:",
    "  • Efeito da experiência (idade)",
    "  • Diferenças de gênero (mulheres têm mais educação em média)",
    "  • Diferenças setoriais (setores pagam diferentemente)",
    "  • Localização geográfica (RM vs não RM)",
    "- Ao controlar por outras variáveis, isolamos o efeito 'puro' da educação",
    "",
    "2.c) Qual a importância de controlar idade, sexo e setor?",
    "- Idade: controla curva de experiência/idade no mercado",
    "- Sexo: controla diferenças salariais de gênero (discriminação/segmentação)",
    "- Setor: controla diferenças setoriais na remuneração (tecnologia vs agro)",
    "- Sem controle: coeficientes são viesados por variáveis omitidas",
    "- Com controle: estimamos efeito causal mais próximo",
    "",
    "CONCLUSÃO: Coeficientes mudam significativamente, mostrando a importância",
    "do controle multivariado para inferência causal."
])

QUESTÃO 3 – Experiência e Ciclo de Vida

In [27]:
q3_code = [
    "CÓDIGO:",
    "# Já estimado no modelo_q2",
    "b_idade = modelo_q2.params['idade']",
    "b_idade2 = modelo_q2.params['I(idade ** 2)']",
    "idade_max = -b_idade / (2 * b_idade2)"
]

b_idade = modelo_q2.params['idade']
b_idade2 = modelo_q2.params['I(idade ** 2)']
idade_max = -b_idade / (2 * b_idade2) if b_idade2 != 0 else np.nan

q3_lines = q3_code + [""] + [
    "RESULTADOS E INTERPRETAÇÃO:",
    "",
    f"Coeficientes: idade = {b_idade:.6f}, idade² = {b_idade2:.6f}",
    "",
    "3.a) O que a forma quadrática sugere sobre o ciclo de vida dos salários?",
    "- Coeficiente positivo em idade: salários aumentam com experiência",
    "- Coeficiente negativo em idade²: taxa de aumento diminui com o tempo",
    "- Forma de U invertido: salários crescem, atingem pico e depois declinam",
    "- Padrão consistente com teoria do capital humano:",
    "  • Fase inicial: aprendizado e acumulação de experiência",
    "  • Fase madura: máxima produtividade",
    "  • Fase final: possível obsolescência skills/desinvestimento",
    "",
    f"3.b) Em que idade o rendimento máximo é alcançado?",
    f"- Fórmula: idade_max = -b_idade / (2 * b_idade2)",
    f"- Cálculo: -({b_idade:.6f}) / (2 * {b_idade2:.6f})",
    f"- Idade de máximo rendimento ≈ {idade_max:.1f} anos",
    "",
    "ANÁLISE:",
    f"- Até ~{idade_max:.0f} anos: salários crescem em média",
    f"- Após ~{idade_max:.0f} anos: salários tendem a declinar",
    "- Padrão típico de ciclo de vida profissional",
    "",
    "CONCLUSÃO: Relação não-linear entre idade e salários, com formato",
    "quadrático capturando ciclo de vida completo."
]

QUESTÃO 4 – Testando Heterocedasticidade

In [28]:
q4_code = [
    "CÓDIGO:",
    "from statsmodels.stats.diagnostic import het_breuschpagan",
    "lm_stat, lm_pvalue, f_stat, f_pvalue = het_breuschpagan(",
    "    modelo_q2.resid, modelo_q2.model.exog",
    ")"
]

lm_stat, lm_pvalue, f_stat, f_pvalue = het_breuschpagan(modelo_q2.resid, modelo_q2.model.exog)

q4_lines = q4_code + [""] + [
    "RESULTADOS E INTERPRETAÇÃO:",
    "",
    f"Estatística LM = {lm_stat:.4f}",
    f"P-valor = {lm_pvalue:.4f}",
    "",
    "4.a) Qual a hipótese nula do teste de Breusch-Pagan?",
    "- H₀: Homocedasticidade (variância constante dos resíduos)",
    "- H₁: Heterocedasticidade (variância não constante dos resíduos)",
    "- Testa se variância dos erros depende das variáveis explicativas",
    "",
    "4.b) O resultado indica heterocedasticidade?",
    f"- P-valor = {lm_pvalue:.4f}",
    f"- Nível de significância: α = 0.05",
    f"- Decisão: {'REJEITAR H₀' if lm_pvalue < 0.05 else 'NÃO REJEITAR H₀'}",
    f"- Conclusão: {'HÁ' if lm_pvalue < 0.05 else 'NÃO HÁ'} evidências de heterocedasticidade",
    "",
    "4.c) O que isso implica para os erros-padrão tradicionais?",
    "- Com heterocedasticidade:",
    "  • Erros-padrão tradicionais são viesados/inconsistentes",
    "  • Testes t e F perdem validade (tamanho do teste incorreto)",
    "  • Intervalos de confiança ficam distorcidos",
    "  • Inferências estatísticas ficam comprometidas",
    "- Solução: usar erros-padrão robustos à heterocedasticidade",
    "",
    "CONCLUSÃO: Presença de heterocedasticidade requer correção nos",
    "erros-padrão para inferências válidas."
]

QUESTÃO 5 – Erros-Padrão Robustos

In [29]:
q5_code = [
    "CÓDIGO:",
    "modelo_robusto = smf.ols(formula=formula_prof, data=banco).fit(cov_type='HC0')",
    "# HC0 = White's heteroskedasticity-consistent estimator"
]

modelo_robusto = smf.ols(formula=formula_prof, data=banco).fit(cov_type='HC0')

q5_lines = q5_code + [""] + [
    "RESULTADOS E INTERPRETAÇÃO:",
    "",
    "5.a) Compare com os erros-padrão da regressão tradicional:",
    ""
]

# Comparação detalhada
variaveis_chave = ['idade', 'I(idade ** 2)', 'mulher', 'rm', 'nao_branco', 'urbano', 'form']
q5_lines.append("Variável | SE Tradicional | SE Robusto | Variação | Significância")
q5_lines.append("---------|----------------|-------------|-----------|-------------")

for var in variaveis_chave:
    if var in modelo_q2.params.index:
        se_trad = modelo_q2.bse[var]
        se_rob = modelo_robusto.bse[var]
        diff_pct = ((se_rob - se_trad) / se_trad) * 100
        sig_trad = modelo_q2.pvalues[var] < 0.05
        sig_rob = modelo_robusto.pvalues[var] < 0.05

        simbolo_trad = "✓" if sig_trad else "✗"
        simbolo_rob = "✓" if sig_rob else "✗"

        q5_lines.append(f"{var:8} | {se_trad:6.4f} ({simbolo_trad}) | {se_rob:6.4f} ({simbolo_rob}) | {diff_pct:+5.1f}% | {'MANTÉM' if sig_trad == sig_rob else 'ALTERA'}")

q5_lines.extend([
    "",
    "5.b) Algum coeficiente deixou de ser significativo?",
])

# Verificar mudanças de significância
mudancas = []
for var in variaveis_chave:
    if var in modelo_q2.params.index:
        sig_antes = modelo_q2.pvalues[var] < 0.05
        sig_depois = modelo_robusto.pvalues[var] < 0.05
        if sig_antes != sig_depois:
            mudanca = "PERDEU significância" if sig_antes and not sig_depois else "GANHOU significância"
            mudancas.append(f"  • {var}: {mudanca}")

if mudancas:
    q5_lines.extend(mudancas)
else:
    q5_lines.append("  • Nenhuma mudança na significância estatística das variáveis principais")

q5_lines.extend([
    "",
    "5.c) O que aprendemos sobre robustez?",
    "- Erros robustos são geralmente MAIORES que os tradicionais",
    "- São mais conservadores (protegem contra heterocedasticidade)",
    "- Mantêm validade dos testes mesmo com variância não constante",
    "- Em amostras grandes, são preferíveis por serem consistentes",
    "- Podem alterar conclusões sobre significância estatística",
    "",
    "CONCLUSÃO: Uso de erros robustos aumenta a confiabilidade e",
    "robustez das inferências estatísticas."
])

QUESTÃO 6 – Previsão de Salário-Hora

In [30]:
q6_code = [
    "CÓDIGO:",
    "novo_individuo = pd.DataFrame({",
    "    'idade': [30],",
    "    'mulher': [1],",
    "    'instrucao': pd.Categorical([6]),",
    "    'ramo': pd.Categorical([8]),",
    "    'rm': [1],",
    "    'nao_branco': [1],",
    "    'urbano': [1],",
    "    'form': [1]",
    "})",
    "pred_log = modelo_q2.predict(novo_individuo)[0]",
    "pred_salario = np.exp(pred_log)"
]

novo_individuo = pd.DataFrame({
    'idade': [30],
    'mulher': [1],
    'instrucao': pd.Categorical([6], categories=banco['instrucao'].cat.categories),
    'ramo': pd.Categorical([8], categories=banco['ramo'].cat.categories),
    'rm': [1],
    'nao_branco': [1],
    'urbano': [1],
    'form': [1]
})

pred_log = modelo_q2.predict(novo_individuo)[0]
pred_salario = np.exp(pred_log)
media_amostral = np.exp(banco['logsalh']).mean()

q6_lines = q6_code + [""] + [
    "RESULTADOS E INTERPRETAÇÃO:",
    "",
    "CARACTERÍSTICAS DO INDIVÍDUO:",
    "• Idade: 30 anos",
    "• Gênero: Mulher",
    "• Educação: Superior completo (instrucao=6)",
    "• Setor: Informação/Comunicação (ramo=08)",
    "• Local: Região Metropolitana (rm=1)",
    "• Cor/Raça: Não branca (nao_branco=1)",
    "• Área: Urbana (urbano=1)",
    "• Vínculo: Formal (form=1)",
    "",
    f"6.a) Log-salário previsto: {pred_log:.4f}",
    f"6.b) Salário-hora previsto: R$ {pred_salario:.2f}",
    f"6.c) Média da amostra: R$ {media_amostral:.2f}",
    f"6.d) Diferença: R$ {pred_salario - media_amostral:+.2f}",
    f"6.e) Posição relativa: {(pred_salario/media_amostral - 1)*100:+.1f}% da média",
    "",
    "6.f) O que isso mostra sobre o uso de regressões para previsão?",
    "- Regressão permite prever resultados para combinações específicas",
    "- Fatores POSITIVOS para este indivíduo:",
    "  • Educação superior completa (+ efeito forte)",
    "  • Setor de tecnologia (+ bem remunerado)",
    "  • Região metropolitana (+ oportunidades)",
    "  • Formalização (+ proteção/benefícios)",
    "- Fatores NEGATIVOS:",
    "  • Gênero feminino (- discriminação/segmentação)",
    "  • Cor não branca (- discriminação)",
    "- Salário acima da média apesar dos fatores negativos",
    "- Modelo captura trade-offs entre características",
    "",
    "CONCLUSÃO: Regressões são ferramentas poderosas para previsão",
    "e análise de determinantes salariais, permitindo isolamento",
    "de efeitos parciais de cada característica."
]

MONTAGEM DO PDF

In [31]:
pages = [
    ('Disciplina : Modelagem Estatística - T01 (2025.2) - LISTA 01', [
    'Aluno : Bárbara Junqueira dos Santos',
    'Matrícula : 20252001826',
    'link do colab: https://colab.research.google.com/drive/1O5hS-31GQPMuhSdmfuYdh5AhF-ACARCJ#scrollTo=PQXmsPBUL6z9',
    '',
        'QUESTÕES RESOLVIDAS:',
        '1. Regressão Simples: Educação e Salário',
        '2. Regressão Múltipla: Controlando por Outras Variáveis',
        '3. Experiência e Ciclo de Vida',
        '4. Testando Heterocedasticidade',
        '5. Erros-Padrão Robustos',
        '6. Previsão de Salário-Hora',
        '',
        'BASE DE DADOS:',
        '• PNAD Contínua - Minas Gerais (4º trimestre/2023)',
        '• Amostra: 2.000 indivíduos',
        '• Variável dependente: log(salário-hora)',
        '',
        'METODOLOGIA:',
        '• Modelos OLS com statsmodels',
        '• Testes de heterocedasticidade',
        '• Erros-padrão robustos (HC0)',
        '• Previsões pontuais'
    ]),
    ('QUESTÃO 1 - Educação e Salário', q1_lines),
    ('QUESTÃO 2 - Regressão Múltipla', q2_lines),
    ('QUESTÃO 3 - Ciclo de Vida', q3_lines),
    ('QUESTÃO 4 - Heterocedasticidade', q4_lines),
    ('QUESTÃO 5 - Erros Robustos', q5_lines),
    ('QUESTÃO 6 - Previsão Salarial', q6_lines)
]

print(f"Gerando PDF com {len(pages)} páginas...")

with PdfPages(OUTPUT_PDF) as pdf:
    for title, content in pages:
        page_text(pdf, title, content)

print('✅ PDF detalhado gerado com sucesso em:', OUTPUT_PDF)
print(f'📄 Total de páginas: {len(pages)}')
print('🎯 Características: Interpretações detalhadas + código por questão')

Gerando PDF com 7 páginas...
✅ PDF detalhado gerado com sucesso em: /content/lista1_report_detalhado.pdf
📄 Total de páginas: 7
🎯 Características: Interpretações detalhadas + código por questão
